# **PRACTICAL EXERCISE**

In [ ]:
!pip install PyPDF2 faiss-cpu sentence-transformers nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 13.5 MB/s eta 0:00:00


In [ ]:
import os
import PyPDF2
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pdf_path = '/content/Law File.PDF'
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text() + ' '
    return text

pdf_text = extract_text_from_pdf(pdf_path)
print("Text extraction complete.")

Text extraction complete.


In [ ]:
sentences = sent_tokenize(pdf_text)
print(f"Total sentences extracted: {len(sentences)}")

Total sentences extracted: 409


In [ ]:
def create_chunks(sentences, chunk_size=5):
    chunks = []
    for i in range(0, len(sentences), chunk_size):
        chunk = ' '.join(sentences[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

text_chunks = create_chunks(sentences, chunk_size=5)
print(f"Total text chunks created: {len(text_chunks)}")

Total text chunks created: 82


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(text_chunks, show_progress_bar=True)
print(f"Embeddings shape: {embeddings.shape}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Embeddings shape: (82, 384)


In [ ]:
embedding_dim = embeddings.shape[1]
print(f"Embedding dimension: {embedding_dim}")

Embedding dimension: 384


In [ ]:
index = faiss.IndexFlatL2(embedding_dim)
print(f"Is the index trained? {index.is_trained}")

Is the index trained? True


In [ ]:
# Convert embeddings to float32, as required by FAISS
embeddings = embeddings.astype('float32')

# Add embeddings to the index
index.add(embeddings)
print(f"Number of vectors in the index: {index.ntotal}")

Number of vectors in the index: 82


In [ ]:
faiss.write_index(index, 'faiss_index.index')

In [ ]:
# To load the index later
index = faiss.read_index('faiss_index.index')

In [ ]:
def query_faiss_index(query, model, index, text_chunks, top_k=5):
    # Generate embedding for the query
    query_embedding = model.encode([query]).astype('float32')

    # Search the index
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve the most similar text chunks
    results = []
    for idx, distance in zip(indices[0], distances[0]):
        results.append({
            'text': text_chunks[idx],
            'distance': distance
        })
    return results

sample_query = "Who is Bhaskar lal sharma?"

results = query_faiss_index(sample_query, model, index, text_chunks, top_k=3)

print("Top 3 similar text chunks:")
for i, result in enumerate(results, 1):
    print(f"\nResult {i}:")
    print(f"Distance: {result['distance']}")
    print(f"Text: {result['text']}")

Top 3 similar text chunks:

Result 1:
Distance: 0.7895253896713257
Text: BHASKAR LAL SHARMA - SMT. VIMLA  SHARMA                            -
FATHER IN LAW            (MOTHER IN LAW)Bhaskar Lal Sharma & Anr vs Monica on 27 July, 2009
Indian Kanoon - http://indiankanoon.org/doc/1675377/ 10 1. He threatened the Complainant to 1. She sent only two unmarried girls finish her relationship
with Mr. for Shagun instead of seven (page Vikas Sharma as she was trying to 42) control their
house, children and the business (page 57)
2. He offered divorce by mutual 2.

Result 2:
Distance: 0.8138992786407471
Text: They thereafter left for Lubumbashi, Conga, Africa. They stayed there for 2 months in their matrimonial home. The relationship between the parties was
cordial during that period.Bhaskar Lal Sharma & Anr vs Monica on 27 July, 2009
Indian Kanoon - http://indiankanoon.org/doc/1675377/ 1 Monica came back to India on 5.4.2004. She stayed at her matrimonial home at Lajpat Nagar, New
Delhi till 10.